__Model Mapping__

In this tutorial, we'll parameterize a simple model and use PyAutoFit to map its parameters to a model instance, which we'll ultimately need to fit to data.

In [ ]:
import autofit as af
import matplotlib.pyplot as plt

To get a feeling for our model lets look at the data we'll ultimately be fitting, a 1D Gaussian.

You need to change the path below to the chapter 1 directory so we can load the dataset.

In [ ]:
chapter_path = "/home/jammy/PycharmProjects/PyAuto/autofit_workspace/howtofit/chapter_1_introduction/"

Lets setup the config files for this tutorial. We'll cover what configs do as we go through the tutorials.

In [ ]:
af.conf.instance = af.conf.Config(config_path=chapter_path + "/config")

The dataset path specifies where the dataset is located, this time in the directory 'chapter_path/dataset'. We'll load the example dataset containing one Gaussian.

In [ ]:
dataset_path = chapter_path + "dataset/gaussian_x1/"

We now load this dataset from .fits files and create an instance of a 'Dataset' object. Checkout the file

'autofit_workspace/howtofit/chapter_1_introduction/tutorial_1_model_mapping/model/src/dataset/dataset.py'

for a description of the Dataset class.

This file is in the folder 'src',  signifying this code would be in the project's source-code. An aim of these tutorials is clearly define what belongs in the source code and what belongs in the code we use to perform model-fitting, e.g. these tutorial scripts. So, make sure to take note of what goes in the source code!

In [ ]:
from howtofit.chapter_1_introduction.tutorial_1_model_mapping.src.dataset import (
    dataset as ds,
)

dataset = ds.Dataset.from_fits(
    data_path=dataset_path + "data.fits", noise_map_path=dataset_path + "noise_map.fits"
)

Lets plot the Gaussian. In this tutorial we'll just use a Matplotlib directly, but in tutorial 2 we'll expand our plotting tools.

In [ ]:
plt.plot(dataset.xvalues, dataset.data)
plt.show()

Its not until tutorial 3 that we'll actually fit this image with a model. But its worth us looking at it now so we can understand the model we're going to fit. So what is the model?

Clearly, its a one-dimensional Gaussian defined as:

g(x, I, sigma) = (I / (sigma * sqrt(2*pi)) * exp (-0.5 * (x / sigma)^2)

Where:

x - Is x-axis coordinate where the Gaussian is evaluated.
I - Describes the intensity of the Gaussian.
sigma - Describes the size of the Gaussian.

This simple equation describes our model - a 1D Gaussian - and it has 3 parameters, (x, I, sigma). Using different values of these 3 parameters we can describe *any* possible 1D Gaussian.

At its core, PyAutoFit is all about making it simple to define a model and straight forwardly map a set of input parameters to the model.

So lets go ahead and create our model of a 1D Gaussian. Take a look at the file

'autofit_workspace/howtofit/chapter_1_introduction/tutorial_1_model_mapping/model/src/model/gaussian.py'.

Here we define our 1D Gaussian model using the code:

In [ ]:
class Gaussian:
    def __init__(
        self,
        centre=0.0,  # <- PyAutoFit recognises these constructor arguments
        intensity=0.1,  # <- are the Gaussian's model parameters.
        sigma=0.01,
    ):
        self.centre = centre
        self.intensity = intensity
        self.sigma = sigma

The class's format is how PyAutoFit requires the components of a model to be written, where:

- The name of the class is the name of the model component, in this case, "Gaussian".
- The input arguments of the constructor are the model parameters which we will ultimately fit for.
- The default values of the input arguments tell PyAutoFit whether a parameter is a single-valued float (e.g. like the intensity and sigma) or a multi-valued tuple. For the Gaussian class, no input parameters are a tuple, however we will show an example of a tuple input in a later tutorial).

By writing a model component in this way, we can use the Python class to set it up as model in PyAutoFit.

In [ ]:
from howtofit.chapter_1_introduction.tutorial_1_model_mapping.src.model import gaussian

model = af.PriorModel(gaussian.Gaussian)

The model is what PyAutoFit calls a PriorModel - we'll explain the name below.